# Train the model

In [2]:
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, Embedding, Dense
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
from keras.models import load_model
import numpy as np
import sentencepiece as spm

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarn

In [3]:
batch_size = 64
epochs = 100
hidden_dims = 256

data_file = "jpn.txt"
enc_input_tokens = []
dec_input_tokens = []
dec_target_tokens = []
start_token_id = 1
end_token_id = 2
pad_token_id = 3

with open(data_file, "r", encoding="utf-8") as f:
    lines_list = f.read().split("\n")

tokenizer = spm.SentencePieceProcessor()
tokenizer.Load("sentencepiece/spm_for_nmt.model")

for line in lines_list:
    #for the last black data, we need to skip
    if line == "":
        break
    source_text, target_text = line.split("\t")
    tokenized_source_text = tokenizer.EncodeAsPieces(source_text)
    #test = tokenizer.EncodeAsIds(source_text)
    #print(test)
    tokenized_target_text = tokenizer.EncodeAsPieces(target_text)
    
    int_tokenized_source = []
    int_tokenized_input_target = []
    int_tokenized_output_target = []
    for token in tokenized_source_text:
        int_tokenized_source.append(tokenizer.piece_to_id(token))
    for i, token in enumerate(tokenized_target_text):
        if i == 0:
            int_tokenized_input_target.append(start_token_id)
            continue
        int_tokenized_input_target.append(tokenizer.piece_to_id(token))
        int_tokenized_output_target.append(tokenizer.piece_to_id(token))
        
    int_tokenized_output_target.append(end_token_id)
    
    if len(int_tokenized_output_target) != len(int_tokenized_input_target):
        print("Error")
        
    enc_input_tokens.append(int_tokenized_source)
    dec_input_tokens.append(int_tokenized_input_target)
    dec_target_tokens.append(int_tokenized_output_target)

len(enc_input_tokens), len(dec_input_tokens), len(dec_target_tokens)

(45093, 45093, 45093)

In [4]:
max_enc_seq = max([len(i) for i in enc_input_tokens])
max_dec_seq = max([len(i) for i in dec_input_tokens])

max_enc_seq, max_dec_seq

(128, 61)

In [5]:
from tqdm import tqdm

def pad_or_truncate_inputs(data, max_len):
    new_data = []
        
    for sample in tqdm(data):
        if len(sample) >= max_len:
            tmp = sample[:max_len]
        else:
            tmp = sample
            num_of_pads_needed = max_len - len(sample)
            for _ in range(num_of_pads_needed):
                tmp.append(pad_token_id)
                
        new_data.append(tmp)
        
    return new_data

In [6]:
enc_input_tokens = pad_or_truncate_inputs(enc_input_tokens, max_enc_seq)
dec_input_tokens = pad_or_truncate_inputs(dec_input_tokens, max_dec_seq)
dec_target_tokens = pad_or_truncate_inputs(dec_target_tokens, max_dec_seq)

len(enc_input_tokens), len(dec_input_tokens), len(dec_target_tokens)

100%|██████████| 45093/45093 [00:00<00:00, 79166.50it/s]


(45093, 45093, 45093)

In [7]:
enc_input_tokens[100]

[1147,
 523,
 599,
 395,
 161,
 107,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3]

In [8]:
print(dec_input_tokens[0],"\n", dec_target_tokens[0])

[1, 168, 421, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 
 [168, 421, 6, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [9]:
np.random.seed(1234)

def shuffle_dataset_and_split_into_train_test(enc_input, dec_input, dec_target, test_ratio=0.2):
    dataset_list = list(zip(enc_input, dec_input, dec_target))
    np.random.shuffle(dataset_list)
    split_point = int(len(enc_input) * test_ratio)
    test = dataset_list[:split_point]
    train = dataset_list[split_point:]
    return train, test

In [10]:
train, test = shuffle_dataset_and_split_into_train_test(enc_input_tokens, dec_input_tokens, dec_target_tokens)
len(train), len(test), len(train[0]), len(test[0])

(36075, 9018, 3, 3)

In [11]:
len(train[0][0]), len(train[0][1]), len(train[0][2])

(128, 61, 61)

In [10]:
def generate_data(data_list, batch_size, shuffle=False):
    while True:
        if shuffle:
            np.random.shuffle(data_list)
            
        for i in range(0, len(data_list), batch_size):
            enc_input_list = []
            dec_input_list = []
            dec_target_list = []
            batch_list_inside_tuples = data_list[i: i + batch_size]
            
            for sample in batch_list_inside_tuples:
                e_inp, d_inp, d_tar = sample[0], sample[1], sample[2]
                enc_input_list.append(e_inp)
                dec_input_list.append(d_inp)
                dec_target_list.append(d_tar)
            np_batch_enc_input = np.vstack(enc_input_list)
            np_batch_dec_input = np.vstack(dec_input_list)
            np_batch_dec_target = np.vstack(dec_target_list)
            np_batch_dec_target_one_hot = to_categorical(np_batch_dec_target, num_classes=vocab_size)
            ##input values are inside of [], and the rest is output value
            yield [np_batch_enc_input, np_batch_dec_input], np_batch_dec_target_one_hot

In [11]:
train_on_batch = generate_data(train, batch_size, shuffle=True)
test_on_batch = generate_data(test, batch_size)
train_steps_per_epoch = len(train) // batch_size
test_steps_per_epoch = len(test) // batch_size
train_steps_per_epoch, test_steps_per_epoch

(563, 140)

In [12]:
vocab_size = 8000
hidden_dims = 256
emb_dims = 300

##training network architecture
enc_inputs = Input(shape=(max_enc_seq,))
enc_emb = Embedding(vocab_size, emb_dims)(enc_inputs)
enc = CuDNNLSTM(hidden_dims, return_state=True)
_, state_h, state_c = enc(enc_emb)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_emb = Embedding(vocab_size, emb_dims)(dec_inputs)
#return_state is used when the model inferences
dec = CuDNNLSTM(hidden_dims, return_sequences=True, return_state=True)
dec_outputs, _, _ = dec(dec_emb, initial_state=enc_states)
dec_dense = Dense(8000, activation="softmax")
dec_outputs = dec_dense(dec_outputs)

model = Model([enc_inputs, dec_inputs], dec_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 128, 300)     2400000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    2400000     input_2[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm

In [13]:
plot_model(model, to_file="seq2seq_for_correct_format.png", show_shapes=True)

In [14]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])

In [15]:
earlystopping = EarlyStopping(monitor="val_loss", patience=2, verbose=1, mode="auto")

model.fit_generator(
    generator=train_on_batch,
    steps_per_epoch=train_steps_per_epoch,
    epochs=epochs,
    verbose=1,
    callbacks=[earlystopping],
    validation_data=test_on_batch,
    validation_steps=test_steps_per_epoch
)

Epoch 1/100
563/563 [==============================] - 56s 100ms/step - loss: 1.2054 - acc: 0.8357 - val_loss: 0.9294 - val_acc: 0.8509
Epoch 2/100
563/563 [==============================] - 50s 88ms/step - loss: 0.8705 - acc: 0.8576 - val_loss: 0.8368 - val_acc: 0.8616
Epoch 3/100
563/563 [==============================] - 44s 77ms/step - loss: 0.7911 - acc: 0.8667 - val_loss: 0.7759 - val_acc: 0.8686
Epoch 4/100
563/563 [==============================] - 41s 73ms/step - loss: 0.7381 - acc: 0.8727 - val_loss: 0.7351 - val_acc: 0.8739
Epoch 5/100
563/563 [==============================] - 42s 74ms/step - loss: 0.6951 - acc: 0.8773 - val_loss: 0.7078 - val_acc: 0.8770
Epoch 6/100
563/563 [==============================] - 42s 74ms/step - loss: 0.6630 - acc: 0.8806 - val_loss: 0.6847 - val_acc: 0.8795
Epoch 7/100
563/563 [==============================] - 42s 74ms/step - loss: 0.6341 - acc: 0.8839 - val_loss: 0.6683 - val_acc: 0.8818
Epoch 8/100
563/563 [==============================] -

In [182]:
model.save("seq2seq_modified.h5", include_optimizer=False)
print("The model is saved!")

/usr/local/lib/python3.5/dist-packages/keras/engine/network.py:877: UserWarning: Layer cu_dnnlstm_8 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'cu_dnnlstm_7/strided_slice_16:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'cu_dnnlstm_7/strided_slice_17:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


The model is saved!


# Inference by the model

In [183]:
model = load_model("seq2seq_modified.h5", compile=False)

In [16]:
##inference network architecture
enc_model = Model(enc_inputs, enc_states)
enc_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 128, 300)          2400000   
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     [(None, 256), (None, 256) 571392    
Total params: 2,971,392
Trainable params: 2,971,392
Non-trainable params: 0
_________________________________________________________________


In [17]:
plot_model(enc_model, to_file="modified_enc_model_for_inference.png", show_shapes=True)

In [1]:
dec_input_state_h = Input(shape=(hidden_dims,))
dec_input_state_c = Input(shape=(hidden_dims,))
dec_input_states = [dec_input_state_h, dec_input_state_c]
dec_outputs, state_h, state_c = dec(dec_emb(dec_inputs), initial_state=dec_input_states)
dec_states = [state_h, state_c]
dec_outputs = dec_dense(dec_outputs)
dec_model = Model([dec_inputs] + dec_input_states,
                 [dec_outputs] + dec_states)
dec_model.summary()

NameError: name 'Input' is not defined

In [19]:
plot_model(enc_model, to_file="modified_dec_model_for_inference.png", show_shapes=True)

In [27]:
input_text = "We listened to her for some time."
tokenized_ids = tokenizer.EncodeAsIds(input_text)
tokenized_ids = np.array(tokenized_ids).reshape(1, len(tokenized_ids)).tolist()
enc_input = pad_or_truncate_inputs(tokenized_ids, max_enc_seq)
enc_input = np.array(enc_input).reshape(1, len(tokenized_ids[0]))
enc_states = enc_model.predict(enc_input)
output_seq = []
cur_token = [start_token_id]
#cur_token = np.array(cur_token).reshape(1, len(cur_token)).tolist()
#cur_token = pad_or_truncate_inputs(cur_token, max_dec_seq)
i = 0
while cur_token != end_token_id and i < (max_dec_seq -1):
    i += 1
    #print("i", cur_token)
    dec_inputs = [cur_token] + enc_states
    [out_dist_vec, sh, sc] = dec_model.predict(dec_inputs)
    print(out_dist_vec.shape)
    output_token = np.argmax(out_dist_vec[0,0], axis=-1)
    #print("o", output_token)
    output_seq.append(tokenizer.id_to_piece(int(output_token)))
    cur_token = [output_token]
    #cur_token = np.array(output_token).reshape(1, 1).tolist()
    #cur_token = pad_or_truncate_inputs(cur_token, max_dec_seq)

print(output_seq)

100%|██████████| 1/1 [00:00<00:00, 1297.74it/s]

(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
(1, 1, 8000)
['私', 'も', '彼女', 'いない', 'と', '私', 'も', '彼女', 'いない', 'と', '私', 'も', '彼女', 'いない', 'と', '私', 'も', '彼女', 'いない', 'と', '私', 'も', '彼女', 'いない', 'と', '私', 'も', '彼女', 'いない', 'と', '私', 'も', '彼女', 'いない', 'と', '私', 'も', '彼女', 'いない', 

In [201]:
test_enc = tokenizer.EncodeAsIds(input_text)
test_dec = tokenizer.DecodeIds(test_enc)
test_dec

'閉策役副社長ーション本部 本部長 石渡 元春'

In [48]:
l = [0, 1, 2, 3, 4, 5]

print(np.array(l).reshape(-1, 3).tolist())

[[0, 1, 2], [3, 4, 5]]


In [52]:
test = np.array(tokenized_ids).reshape(1, len(tokenized_ids)).tolist()
print(test)

[[7, 76, 2282, 1404, 7, 1127, 99, 7, 5338, 3441]]


In [86]:
tokenizer.id_to_piece(76)

'マ'

In [166]:
tokenizer.EncodeAsPieces('マネージャー 森川 卓也')

['▁', 'マ', 'ネー', 'ジャー', '▁', '森', '川', '▁', '卓', '也']